# YandexART в DataSphere

Вы можете обращаться к модели YandexART из ноутбуков DataSphere через API. 

**API YandexART находится на стадии Preview.**

## Содержание

1. [Установите зависимости](#section-id1).
2. [Настройте подключение к облаку](#section-id2).
3. [Обратитесь к модели](#section-id3).
4. [Примеры промптов для YandexART](#section-id4).

<a id='section-id1'></a>
## Установите зависимости 
После выполнения ячейки перезагрузите ядро (**Kernel -> Reset Kernel**).После выполнения ячейки перезагрузите ядро (**Kernel -> Reset Kernel**).

In [ ]:
# Устанавливаем пакеты, необходимые для обращения к YandexART

%pip uninstall jwt
%pip install PyJWT -U

In [ ]:
import requests
import json
import time
import jwt
import base64
import os

<a id='section-id2'></a>
## Настройте подключение к облаку

Чтобы обратиться к API YandexART, вам потребуется сервисный аккаунт с ролью `ai.imageGeneration.user` в облаке.
1. Создайте сервисный аккаунт, как описано в [инструкции](https://cloud.yandex.ru/docs/iam/operations/sa/create).
2. В ячейке ниже укажите идентификатор сервисного аккаунта.

In [ ]:
service_account_id = "идентификатор_сервисного_аккаунта"

3. [Создайте](https://cloud.yandex.ru/docs/iam/operations/authorized-key/create) авторизованный ключ для сервисного аккаунта. 
4. Сохраните значение ключа в секрете `private-key`. [Как создать секрет](https://cloud.yandex.ru/docs/datasphere/operations/data/secrets).
5. Идентификатор ключа укажите в ячейке ниже.

In [ ]:
key_id = "идентификатор_ключа"
private_key = os.environ['private-key']

7. Получите IAM-токен для сервисного аккаунта.

In [ ]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
token = x['iamToken']

<a id='section-id3'></a>
## Обратитесь к модели

Обращение к моделям осуществляется по URI. Дополнительная информация моделях YandexART доступна [в документации](https://yandex.cloud/ru/docs/foundation-models/concepts/yandexart/).

Создадим счетчик, чтобы сохранять сгенерированные изображения под разными именами.

In [ ]:
counter = 0

**Код ниже можно исполнять неограниченное число раз. Предыдущие генерации будут сохраняться в текущем каталоге.**

In [ ]:
# Адрес для обращения к модели 
url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/imageGenerationAsync'

data = {}

# Указываем URI модели. Замените <идентификатор_каталога> своим значением
data['modelUri'] = "art://<идентификатор_каталога>/yandex-art/latest"

Сгененировать разные изображения с одной и той же инструкцией можно, если изменять значение праметра `seed`.

In [ ]:
# Настраиваем дополнительные параметры модели
data['generationOptions'] = {'seed': 12345678910}

# Указываем контекст для модели
data['messages'] = [
    {
        "weigth": 1,
        "text": "узор из цветных пастельных суккулентов разных сортов, hd full wallpaper, четкий фокус, множество сложных деталей, глубина кадра, вид сверху"
    },
]

# Получаем ответ модели
response = requests.post(url, headers={'Authorization': 'Bearer ' + token}, json = data).json()
response

Генерация изображения может занимать до нескольких минут. Подождите некоторое время и отправьте запрос, чтобы получить результат генерации. Если изображение готово, результат вернется в кодировке Base64 и будет записан в файл image.jpeg.

In [ ]:
seconds = 30

time.sleep(seconds)

In [ ]:
request_id = response.get('id')

# URL для запроса
url = f"https://llm.api.cloud.yandex.net:443/operations/{request_id}"

# Заголовки запроса
headers = {
    "Authorization": f"Bearer {token}"
}

# Отправка GET-запроса
response = requests.get(url, headers=headers)

# Проверка успешного выполнения запроса
if response.status_code == 200:
    # Получение JSON-ответа
    json_response = response.json()
    
    # Извлечение закодированного изображения
    encoded_image = json_response.get('response', {}).get('image')
    
    if encoded_image:
        # Декодирование изображения из base64
        image_data = base64.b64decode(encoded_image)
        
        # Сохранение изображения в файл с именем image{counter}.jpeg
        image_filename = f'image-{counter}.jpeg'
        with open(image_filename, 'wb') as image_file:
            image_file.write(image_data)
        
        print(f"Изображение успешно сохранено: '{image_filename}'")
        
        # Увеличение счетчика
        counter += 1
    else:
        print("Изображение еще не готово.")
else:
    print(f"Ошибка при выполнении запроса: {response.status_code}")

In [ ]:
from IPython.display import Image, display

# Открытие последней сохраненной картинки
image_filename = f'image-{counter - 1}.jpeg'
Image(filename=image_filename)


#### Примеры промтов для решения различные задачи с помощью YandexART доступны в [библиотеке промтов](https://yandex.cloud/ru/docs/foundation-models/prompts/yandexart)